In [1]:
# ==============================================
# WEATHER EFFECTS ON SALES
# This analysis examines how weather variables (temperature and rainfall)
# affect daily grocery sales, controlling for promotions and holidays.
# One-hot encoding is used for categorical weather variables.
# Baseline categories: Normal Temperature, No Rain
# ==============================================

# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 2. LOAD DATA
# Load sales and weather data
df = pd.read_csv("ecuador_weather_data.csv")

# 3. FEATURE ENGINEERING

# --- Compute average daily temperature ---
df['temperature_2m_avg'] = (df['temperature_2m_max'] + df['temperature_2m_min']) / 2

# --- Categorize temperature into Low / Normal / High based on ±8°C from mean ---
avg_temp = df['temperature_2m_avg'].mean()
df['temp_variation'] = pd.cut(
    df['temperature_2m_avg'],
    bins=[-np.inf, avg_temp - 8, avg_temp + 8, np.inf],
    labels=['Low Temp', 'Normal Temp', 'High Temp']
)

# --- Categorize rainfall intensity ---
# No Rain = 0 mm
# Very Light Rain = <1.0mm
# Light Rain = 1.0 –10.0 mm
# Moderate Rain = 10 - 30mm
# Heavy Rain = >=30 mm
def categorize_precipitation(precip):
    if precip == 0:
        return 'No Rain'
    elif precip < 10.1:
        return 'Light Rain'
    elif precip < 30.1:
        return 'Moderate Rain'
    else:
        return 'Heavy Rain'

df['rain_intensity'] = df['precipitation_sum'].apply(categorize_precipitation)

# 4. ENCODE CATEGORICAL VARIABLES (ONE-HOT ENCODING)

# --- Temperature dummies ---
# Set "Normal Temp" as baseline (dropped from dummies)
temp_dummies = pd.get_dummies(df['temp_variation'], prefix='temp', drop_first=False)
if 'temp_Normal Temp' in temp_dummies.columns:
    temp_dummies = temp_dummies.drop(columns=['temp_Normal Temp'])
df = pd.concat([df, temp_dummies], axis=1)

# --- Rain intensity dummies ---
# Set "No Rain" as baseline (dropped from dummies)
rain_dummies = pd.get_dummies(df['rain_intensity'], prefix='rain', drop_first=False)
if 'rain_No Rain' in rain_dummies.columns:
    rain_dummies = rain_dummies.drop(columns=['rain_No Rain'])
df = pd.concat([df, rain_dummies], axis=1)

# Drop original categorical columns
df = df.drop(columns=['temp_variation', 'rain_intensity'], errors='ignore')

# 5. DEFINE TARGET AND CONTROL VARIABLES

# Target variable
y = df['sales']

# Ensure holiday indicator is numeric
df['is_holiday'] = df['is_holiday'].astype(int)

# Control variables for regression
control_vars = ['onpromotion', 'is_holiday']

# 6. REGRESSION HELPER FUNCTION
def run_regression(y, X, label):
    """
    Runs an OLS regression and prints the summary table for coefficients.
    """
    X = sm.add_constant(X)  # Add intercept
    model = sm.OLS(y.astype(float), X.astype(float)).fit()
    print(f"\n--- {label} ---")
    print(model.summary().tables[1])  # Show coefficients table
    return model

# 7. RUN REGRESSION MODELS

# --- 7a. Continuous weather variables ---
X_cont = df[['temperature_2m_avg', 'precipitation_sum']]
model_cont = run_regression(y, X_cont, "Continuous Weather Variables (No Controls)")

X_cont_ctrl = df[['temperature_2m_avg', 'precipitation_sum'] + control_vars]
model_cont_ctrl = run_regression(y, X_cont_ctrl, "Continuous Weather Variables (Controlled)")

# --- 7b. Temperature categories (Low / High, baseline = Normal) ---
temp_vars_dummies = [col for col in df.columns if col.startswith('temp_')]
X_temp = df[temp_vars_dummies]
model_temp = run_regression(y, X_temp, "Temperature Categories (No Controls)")

X_temp_ctrl = df[temp_vars_dummies + control_vars]
model_temp_ctrl = run_regression(y, X_temp_ctrl, "Temperature Categories (Controlled)")

# --- 7c. Rainfall intensity categories (Light / Moderate / Heavy, baseline = No Rain) ---
rain_vars_dummies = [col for col in df.columns if col.startswith('rain_')]
X_rain = df[rain_vars_dummies]
model_rain = run_regression(y, X_rain, "Rainfall Intensity (No Rain as Baseline)")

X_rain_ctrl = df[rain_vars_dummies + control_vars]
model_rain_ctrl = run_regression(y, X_rain_ctrl, "Rainfall Intensity (No Rain as Baseline + Controls)")

# 8. CREATE SUMMARY TABLE OF COEFFICIENTS
summary_table = pd.DataFrame({
    "Continuous": model_cont.params,
    "Continuous + Controls": model_cont_ctrl.params,
    "Temp Categories": model_temp.params,
    "Temp + Controls": model_temp_ctrl.params,
    "Rain Categories": model_rain.params,
    "Rain + Controls": model_rain_ctrl.params
}).round(4)

# Print final summary
print("\n============================================================")
print("FINAL SUMMARY COMPARISON (CONTROLLED FOR onpromotion + is_holiday)")
print("BASELINE: Normal Temperature + No Rain")
print("============================================================")
print(summary_table)


--- Continuous Weather Variables (No Controls) ---
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1575.6874      5.903    266.944      0.000    1564.118    1587.256
temperature_2m_avg   -41.8796      0.283   -148.025      0.000     -42.434     -41.325
precipitation_sum      1.3819      0.157      8.807      0.000       1.074       1.689

--- Continuous Weather Variables (Controlled) ---
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1329.8491      5.463    243.445      0.000    1319.143    1340.556
temperature_2m_avg   -39.3310      0.260   -151.067      0.000     -39.841     -38.821
precipitation_sum      1.2714      0.144      8.809      0.000       0.989       1.554
onpromotion           34.10